<a id="1"></a>
# <div style="text-align:center; border-radius:30px 30px; padding:7px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#7E8EF1; overflow:hidden"><b> Ahmed Gaitani Code </b></div>

![mushrooms](https://theminimalistvegan.com/wp-content/uploads/2023/05/irina-iacob-qaUoTVTI6Cc-unsplash-2048x1463.webp)

<a id="1"></a>
# <div style="text-align:center; border-radius:30px 30px; padding:7px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#7E8EF1; overflow:hidden"><b> Import Libraries </b></div>

In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import matthews_corrcoef

<a id="1"></a>
# <div style="text-align:center; border-radius:30px 30px; padding:7px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#7E8EF1; overflow:hidden"><b> Load & Preprocessing Data </b></div>

In [2]:
train_df = pd.read_csv("/kaggle/input/playground-series-s4e8/train.csv", index_col='id')
test_df = pd.read_csv("/kaggle/input/playground-series-s4e8/test.csv", index_col='id')

In [3]:
train_df.head()

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,...,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,,
0,e,8.80,f,s,u,f,a,c,w,4.51,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,a
1,p,4.51,x,h,o,f,a,c,n,4.79,...,NaN,y,o,NaN,NaN,t,z,NaN,d,w
2,e,6.94,f,s,b,f,x,c,w,6.85,...,NaN,s,n,NaN,NaN,f,f,NaN,l,w
3,e,3.88,f,y,g,f,s,NaN,g,4.16,...,NaN,NaN,w,NaN,NaN,f,f,NaN,d,u
4,e,5.85,x,l,w,f,d,NaN,w,3.37,...,NaN,NaN,w,NaN,NaN,f,f,NaN,g,a


In [4]:
cat_cols = ["cap-shape","cap-surface","cap-color","does-bruise-or-bleed","gill-attachment",
                "gill-spacing","gill-color","stem-root","stem-surface","stem-color","veil-type",
                "veil-color","has-ring","ring-type","spore-print-color","spore-print-color",
                "habitat","season"]

In [5]:
for col in cat_cols:
    train_df[col] = train_df[col].fillna('missing')
    test_df[col] = test_df[col].fillna('missing')
    train_df[col] = train_df[col].astype('category')
    test_df[col] = test_df[col].astype('category')

In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3116945 entries, 0 to 3116944
Data columns (total 21 columns):
 #   Column                Dtype   
---  ------                -----   
 0   class                 object  
 1   cap-diameter          float64 
 2   cap-shape             category
 3   cap-surface           category
 4   cap-color             category
 5   does-bruise-or-bleed  category
 6   gill-attachment       category
 7   gill-spacing          category
 8   gill-color            category
 9   stem-height           float64 
 10  stem-width            float64 
 11  stem-root             category
 12  stem-surface          category
 13  stem-color            category
 14  veil-type             category
 15  veil-color            category
 16  has-ring              category
 17  ring-type             category
 18  spore-print-color     category
 19  habitat               category
 20  season                category
dtypes: category(17), float64(3), object(1)
memory usage: 169.5+

In [7]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2077964 entries, 3116945 to 5194908
Data columns (total 20 columns):
 #   Column                Dtype   
---  ------                -----   
 0   cap-diameter          float64 
 1   cap-shape             category
 2   cap-surface           category
 3   cap-color             category
 4   does-bruise-or-bleed  category
 5   gill-attachment       category
 6   gill-spacing          category
 7   gill-color            category
 8   stem-height           float64 
 9   stem-width            float64 
 10  stem-root             category
 11  stem-surface          category
 12  stem-color            category
 13  veil-type             category
 14  veil-color            category
 15  has-ring              category
 16  ring-type             category
 17  spore-print-color     category
 18  habitat               category
 19  season                category
dtypes: category(17), float64(3)
memory usage: 97.1 MB


In [8]:
x = train_df.drop('class', axis=1)
x.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
id,,,,,,,,,,,,,,,,,,,,
0,8.80,f,s,u,f,a,c,w,4.51,15.39,missing,missing,w,missing,missing,f,f,missing,d,a
1,4.51,x,h,o,f,a,c,n,4.79,6.48,missing,y,o,missing,missing,t,z,missing,d,w
2,6.94,f,s,b,f,x,c,w,6.85,9.93,missing,s,n,missing,missing,f,f,missing,l,w
3,3.88,f,y,g,f,s,missing,g,4.16,6.53,missing,missing,w,missing,missing,f,f,missing,d,u
4,5.85,x,l,w,f,d,missing,w,3.37,8.36,missing,missing,w,missing,missing,f,f,missing,g,a


In [9]:
y = train_df['class']

In [10]:
X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

In [11]:
print(X_train.shape)
print(X_val.shape)
print(y_train.shape)
print(y_val.shape)

(2493556, 20)
(623389, 20)
(2493556,)
(623389,)


<a id="1"></a>
# <div style="text-align:center; border-radius:30px 30px; padding:7px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#7E8EF1; overflow:hidden"><b> Build & Train Model </b></div>

In [12]:
clf = LGBMClassifier(objective='binary', metric='binary_error',num_leaves=81,
    learning_rate=0.1, n_estimators=550, max_depth= 9, random_state=21)

In [13]:
clf.fit(X_train,y_train)

[LightGBM] [Info] Number of positive: 1364509, number of negative: 1129047
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.389807 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 949
[LightGBM] [Info] Number of data points in the train set: 2493556, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.547214 -> initscore=0.189421
[LightGBM] [Info] Start training from score 0.189421


LGBMClassifier(max_depth=9, metric='binary_error', n_estimators=550,
               num_leaves=81, objective='binary', random_state=21)

In [14]:
y_pred = clf.predict(X_val)

In [15]:
matthews_corrcoef(y_val, y_pred)

0.9841471696422636

In [16]:
y_pred = clf.predict(test_df)

<a id="1"></a>
# <div style="text-align:center; border-radius:30px 30px; padding:7px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#7E8EF1; overflow:hidden"><b> Create a submission file </b></div>

In [17]:
df_sub = pd.read_csv('/kaggle/input/playground-series-s4e8/sample_submission.csv')

In [18]:
df_sub['class'] = y_pred

In [19]:
df_sub.head()

,id,class
0,3116945,e
1,3116946,p
2,3116947,p
3,3116948,p
4,3116949,e


In [20]:
df_sub.to_csv('submission.csv', index=False)

In [21]:
df_sub.shape

(2077964, 2)

<a id="1"></a>
# <div style="text-align:center; border-radius:30px 30px; padding:7px; color:white; margin:0; font-size:150%; font-family:Pacifico; background-color:#7E8EF1; overflow:hidden"><b> The End ☺ </b></div>